In [2]:
%load_ext autoreload
%autoreload 2

import sympy as sp
from sympy import Derivative, Symbol, IndexedBase, Idx, Sum, Function, Matrix, Add, Eq
from sympy import diff, symbols, solve, linsolve, simplify, poly, pprint, factor, integrate, lambdify
from sympy import cos, sin, pi

# syspath trick
import sys
sys.path.append('../')

from tatarinov.database.singleton import db
db.set_root('../data/triangular_platform')


from tatarinov.utils.jupyter import display_list, display_obj
from tatarinov.core.variables import _omega, _v
from tatarinov.core.tatarinov import TatarinovSystem

from tatarinov.problems.triangular_platform.variables import *
from tatarinov.problems.triangular_platform.structure import *
from tatarinov.problems.triangular_platform.functions import euler, scalar
from tatarinov.utils.sympy import MatrixElDerivative

### Посмотрим, будут ли не зависеть от времени момент при движении по окружности

In [3]:
TWNU3 = db.load('TWNU3').obj
constraints = db.load('constraints').obj
nu1, nu2 = db.load('nu1,nu2(x,y)').obj

Loading from: ../data/triangular_platform/TWNU3.pickle
Loading from: ../data/triangular_platform/constraints.pickle
Loading from: ../data/triangular_platform/nu1,nu2(x,y).pickle


In [35]:
display_list([nu1, nu2])

Eq(nu1(t), sin(alpha(t))*Derivative(y(t), t) + cos(alpha(t))*Derivative(x(t), t))

Eq(nu1(t), sin(alpha(t))*Derivative(y(t), t) + cos(alpha(t))*Derivative(x(t), t))

In [10]:
equations = [TWNU3[2]] + [constraints[2], constraints[3]]

In [11]:
display_list(equations)

Eq(Derivative(nu3(t), t), (-nu1(t)**2*sin(theta0(t))**3 + 3*nu1(t)**2*sin(theta0(t))/4 - nu1(t)**2*sin(3*theta0(t))/4 - 2*nu1(t)*nu2(t)*cos(theta0(t))**3 + 3*nu1(t)*nu2(t)*cos(theta0(t))/2 + nu1(t)*nu2(t)*cos(3*theta0(t))/2 - 4*nu1(t)*nu3(t)*cos(theta0(t))**3 + 3*nu1(t)*nu3(t)*cos(theta0(t))**2 + 9*nu1(t)*nu3(t)*cos(theta0(t)) + nu1(t)*nu3(t)*cos(3*theta0(t)) + nu2(t)**2*sin(theta0(t))**3 - 3*nu2(t)**2*sin(theta0(t))/4 + nu2(t)**2*sin(3*theta0(t))/4 + 4*nu2(t)*nu3(t)*sin(theta0(t))**3 + 3*nu2(t)*nu3(t)*sin(theta0(t))*cos(theta0(t)) - 3*nu2(t)*nu3(t)*sin(theta0(t)) + nu2(t)*nu3(t)*sin(3*theta0(t)) + 4*nu3(t)**2*sin(theta0(t))**3 - 3*nu3(t)**2*sin(theta0(t)) + nu3(t)**2*sin(3*theta0(t)) - 4*sin(theta0(t))*cos(theta0(t))*Derivative(nu1(t), t) + 4*cos(theta0(t))**2*Derivative(nu2(t), t) + 6*cos(theta0(t))*Derivative(nu2(t), t))/(3*cos(theta0(t)) - cos(2*theta0(t)) - 1))

Eq(Derivative(alpha(t), t), nu3(t))

Eq(Derivative(theta0(t), t), (-R*nu3(t)*cos(beta0 - theta0(t)) - d*nu3(t) + nu1(t)*sin(theta0(t)) - nu2(t)*cos(theta0(t)))/d)

In [88]:
const, V = symbols('(const), V')
# w = dynamicsymbols('omega')

subs_dict = {
    x: V*cos(t),
    y: V*sin(t)
}

subs_dict.update({
    nu[1]: nu1.subs(subs_dict).args[1],
    nu[2]: nu2.subs(subs_dict).args[1]
})

subs_dict_const = {
    nu[3]: symbols('C'),
    Derivative(alpha, t): symbols('C')
}

In [85]:
%%time 
equations1 = [simplify(_.subs(subs_dict)) for _ in equations[0::2]]

CPU times: user 25 s, sys: 0 ns, total: 25 s
Wall time: 25 s


In [86]:
equations2 = equations1 + [equations[1]]

In [87]:
display_list(equations2)

Eq(Derivative(nu3(t), t), (-2*V*(1 - Derivative(alpha(t), t))*sin(2*theta0(t))*cos(t - alpha(t)) + 4*V*(1 - Derivative(alpha(t), t))*cos(t - alpha(t))*cos(theta0(t))**2 + 6*V*(1 - Derivative(alpha(t), t))*cos(t - alpha(t))*cos(theta0(t)) + 2*(-V*sin(t)*cos(alpha(t)) + V*sin(alpha(t))*cos(t))**2*cos(theta0(t))**3 - 3*(-V*sin(t)*cos(alpha(t)) + V*sin(alpha(t))*cos(t))**2*cos(theta0(t))/2 - (-V*sin(t)*cos(alpha(t)) + V*sin(alpha(t))*cos(t))**2*cos(3*theta0(t))/2 - 4*(-V*sin(t)*cos(alpha(t)) + V*sin(alpha(t))*cos(t))*nu3(t)*sin(theta0(t))**3 - 3*(-V*sin(t)*cos(alpha(t)) + V*sin(alpha(t))*cos(t))*nu3(t)*sin(theta0(t))*cos(theta0(t)) + 3*(-V*sin(t)*cos(alpha(t)) + V*sin(alpha(t))*cos(t))*nu3(t)*sin(theta0(t)) - (-V*sin(t)*cos(alpha(t)) + V*sin(alpha(t))*cos(t))*nu3(t)*sin(3*theta0(t)) + 4*(-V*sin(t)*cos(alpha(t)) + V*sin(alpha(t))*cos(t))*nu3(t)*cos(theta0(t))**3 - 3*(-V*sin(t)*cos(alpha(t)) + V*sin(alpha(t))*cos(t))*nu3(t)*cos(theta0(t))**2 - 9*(-V*sin(t)*cos(alpha(t)) + V*sin(alpha(t))*cos

Eq(Derivative(theta0(t), t), -(R*nu3(t)*cos(beta0 - theta0(t)) - sqrt(2)*V*sin(t)*cos(theta0(t) + pi/4)*cos(alpha(t)) + sqrt(2)*V*sin(alpha(t))*cos(t)*cos(theta0(t) + pi/4) + d*nu3(t))/d)

Eq(Derivative(alpha(t), t), nu3(t))

In [82]:
# x(t),y(t) ---> [T[1], W[1], Derivative(alpha, t)]

In [90]:
equations3 = [_.subs(subs_dict_const) for _ in equations2]

In [92]:
display_list(equations3)

Eq(Derivative(C, t), (-4*C**2*sin(theta0(t))**3 + 3*C**2*sin(theta0(t)) - C**2*sin(3*theta0(t)) - 4*C*(-V*sin(t)*cos(alpha(t)) + V*sin(alpha(t))*cos(t))*sin(theta0(t))**3 - 3*C*(-V*sin(t)*cos(alpha(t)) + V*sin(alpha(t))*cos(t))*sin(theta0(t))*cos(theta0(t)) + 3*C*(-V*sin(t)*cos(alpha(t)) + V*sin(alpha(t))*cos(t))*sin(theta0(t)) - C*(-V*sin(t)*cos(alpha(t)) + V*sin(alpha(t))*cos(t))*sin(3*theta0(t)) + 4*C*(-V*sin(t)*cos(alpha(t)) + V*sin(alpha(t))*cos(t))*cos(theta0(t))**3 - 3*C*(-V*sin(t)*cos(alpha(t)) + V*sin(alpha(t))*cos(t))*cos(theta0(t))**2 - 9*C*(-V*sin(t)*cos(alpha(t)) + V*sin(alpha(t))*cos(t))*cos(theta0(t)) - C*(-V*sin(t)*cos(alpha(t)) + V*sin(alpha(t))*cos(t))*cos(3*theta0(t)) - 2*V*(1 - C)*sin(2*theta0(t))*cos(t - alpha(t)) + 4*V*(1 - C)*cos(t - alpha(t))*cos(theta0(t))**2 + 6*V*(1 - C)*cos(t - alpha(t))*cos(theta0(t)) + 2*(-V*sin(t)*cos(alpha(t)) + V*sin(alpha(t))*cos(t))**2*cos(theta0(t))**3 - 3*(-V*sin(t)*cos(alpha(t)) + V*sin(alpha(t))*cos(t))**2*cos(theta0(t))/2 - (-V*s

Eq(Derivative(theta0(t), t), -(C*R*cos(beta0 - theta0(t)) + C*d - sqrt(2)*V*sin(t)*cos(theta0(t) + pi/4)*cos(alpha(t)) + sqrt(2)*V*sin(alpha(t))*cos(t)*cos(theta0(t) + pi/4))/d)

True

In [95]:
trigsimp(equations3[0]).doit()

Eq(0, (-4*C**2*sin(theta0(t))**3 + 3*C**2*sin(theta0(t)) - C**2*sin(3*theta0(t)) + 3*C*V*(cos(-t + alpha(t) + 2*theta0(t)) - cos(t - alpha(t) + 2*theta0(t)))/4 + 4*C*V*sin(t - alpha(t))*sin(theta0(t))**3 - 3*C*V*sin(t - alpha(t))*sin(theta0(t)) + C*V*sin(t - alpha(t))*sin(3*theta0(t)) - 4*C*V*sin(t - alpha(t))*cos(theta0(t))**3 + 3*C*V*sin(t - alpha(t))*cos(theta0(t))**2 + 9*C*V*sin(t - alpha(t))*cos(theta0(t)) + C*V*sin(t - alpha(t))*cos(3*theta0(t)) + 2*V**2*sin(t - alpha(t))**2*cos(theta0(t))**3 - 3*V**2*sin(t - alpha(t))**2*cos(theta0(t))/2 - V**2*sin(t - alpha(t))**2*cos(3*theta0(t))/2 - 2*V*(1 - C)*sin(2*theta0(t))*cos(t - alpha(t)) + 4*V*(1 - C)*cos(t - alpha(t))*cos(theta0(t))**2 + 6*V*(1 - C)*cos(t - alpha(t))*cos(theta0(t)))/(-3*cos(theta0(t)) + cos(2*theta0(t)) + 1))